# TP : Implémentation d'un réseau de neurones de type Perceptron multicouche pour un problème de classification
Dans ce TP, vous allez implémenter en Numpy un réseau de neurones de type Perceptron multicouche.


## Configuration
### Si vous utilisez un ordinateur de l'Enseirb:
#### 1) Lancer une session linux (et non pas windows)
#### 2) Aller dans "Applications", puis "Autre", puis "conda_pytorch" (un terminal devrait s'ouvrir)
#### 3) Dans ce terminal, taper la commande suivante pour lancer Spyder :  
`spyder &`  
#### 4) Configurer Spyder en suivant ces instructions [Lien configuration Spyder](https://gbourmaud.github.io/files/configuration_spyder_annotated.pdf).
### Si vous utilisez votre ordinateur personnel, il faudra installer Spyder.

# PARTIE 1 : Exemple "jouet"

### Créer un nouveau script python et copiez/collez le code suivant :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(0)

### Définition d'un exemple jouet à 3 classes en 2D :

In [ ]:
#%% DEFINE AND PLOT DATA
    
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),np.arange(y_min, y_max, h))
    return xx, yy


style_per_class = ['xb', 'or', 'sg']
X = np.array([[1.2, 2.3, -0.7, 3.2, -1.3],[-3.4, 2.8, 1.2, -0.4, -2.3]]).T
X -= X.mean()
X /= X.std()
y = np.array([0,0,1,1,2])


C = len(style_per_class)
N = X.shape[0]
xx, yy = make_meshgrid(X[:,0].ravel(), X[:,1].ravel(), h=0.1)


plt.figure(1)
ax = plt.subplot(111)
ax.set_xlim(xx.min(), xx.max())
ax.set_ylim(yy.min(), yy.max())
plt.grid(True)

for i in range(C):
    x_c = X[(y==i).ravel(),:]
    plt.plot(x_c[:,0],x_c[:,1],style_per_class[i])

plt.pause(0.1)

### Implémentation d'un MLP à une couche cachée :

In [ ]:
class MLP:
    def __init__(self, H):

        self.C = 3
        self.D = 2
        self.H = H
        
        #parameters
        self.W1 = (np.sqrt(6./self.D))*(2*(np.random.uniform(size=(self.D,self.H))-0.5))
        self.b1 = (1./np.sqrt(self.D))*(2*(np.random.uniform(size=(self.H))-0.5))
        self.W3 = (np.sqrt(6./self.H))*(2*(np.random.uniform(size=(self.H,self.C))-0.5))
        self.b3 = (1./np.sqrt(self.H))*(2*(np.random.uniform(size=(self.C))-0.5))
        
        
        #gradients
        self.dl_dW1 = np.zeros_like(self.W1)
        self.dl_db1 = np.zeros_like(self.b1)
        self.dl_dW3 = np.zeros_like(self.W3)
        self.dl_db3 = np.zeros_like(self.b3)
        
    def forward(self,X):
    
        X1 = X.dot(self.W1) + self.b1 #NxH
        X2 = np.maximum(0.,X1) #NxH
        O = X2.dot(self.W3) + self.b3 #NxC
    
        return X,X1,X2,O
    
    def backward(self,dl_dO, O, X2, X1, X0):
        
        #backpropagation of dl_dO through last fully connected layer
        dl_dX2 = ???
        self.dl_dW3 += ???
        self.dl_db3 += ???
        
        #backpropagation of dl_dX2 through ReLU
        dl_dX1 = ???
        
        #backpropagation of dl_dX1 through first fully connected layer
        self.dl_dW1 += ???
        self.dl_db1 += ???
        
        
        return

**Travail : implémenter la fonction `backward` en utilisant les équations obtenues en TP. La sortie du réseau (variable `O`) est-elle utilisée ? Pourquoi ?**

### Définition de la fonction de coût :

In [ ]:
def logsoftmax(x):
    x_shift = x - np.amax(x, axis=1, keepdims=True)
    return x_shift - np.log(np.exp(x_shift).sum(axis=1, keepdims=True))   
    
def softmax(x):
    e_x = np.exp(x - np.amax(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)
    
def multinoulliCrossEntropyLoss(O, y):
    N = y.shape[0]
    P = softmax(O.astype('double'))
    log_p = logsoftmax(O.astype('double'))
    a = log_p[np.arange(N),y]
    l = -a.sum()/N
    dl_do = P
    dl_do[np.arange(N),y] -= 1
    dl_do = dl_do/N
    return (l, dl_do)

### Implémentation de la méthode de descente de gradient avec moment :

In [ ]:
class GradientDescentWithMomentum:
    def __init__(self, model, beta, lr):
        
        self.model = model
        self.beta= beta
        self.lr = lr
        
        #momentum
        self.VW1 = np.zeros_like(self.model.W1)
        self.Vb1 = np.zeros_like(self.model.b1)
        self.VW3 = np.zeros_like(self.model.W3)
        self.Vb3 = np.zeros_like(self.model.b3)
        
    def step(self):
        self.VW1 = self.beta*self.VW1 + (1.0-self.beta)*self.model.dl_dW1
        self.model.W1 -= self.lr*self.VW1

        self.VW3 = self.beta*self.VW3 + (1.0-self.beta)*self.model.dl_dW3
        self.model.W3 -= self.lr*self.VW3
    
        self.Vb1 = self.beta*self.Vb1 + (1.0-self.beta)*self.model.dl_db1
        self.model.b1 -= self.lr*self.Vb1
    
        self.Vb3 = self.beta*self.Vb3 + (1.0-self.beta)*self.model.dl_db3
        self.model.b3 -= self.lr*self.Vb3
    
    def zero_gradients(self):
        self.model.dl_dW1.fill(0.)
        self.model.dl_db1.fill(0.)
        self.model.dl_dW3.fill(0.)
        self.model.dl_db3.fill(0.)

### Ajout d'une fonction d'affichage qui va permettre d'afficher les frontières de décisions :

In [ ]:
def plot_contours(ax, model, xx, yy, **params):
    """Plot the decision boundaries for a classifier.
    Parameters
    ----------
    ax: matplotlib axes object
    model: neural network
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    _,_,_,O = model.forward(np.c_[xx.ravel(), yy.ravel()])
    pred = np.argmax(O, axis=1)
    Z = pred.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    
    return out

### Avant de lancer un apprentissage, il faut choisir les hyper-paramètres de l'architecture et de l'algorithme de descente de gradient (ici nous utilisons une version de l'algorithme de descente de gradient appelée "momentum") :

In [ ]:
#%% HYPERPARAMETERS
H = 300
lr = 1e-2 #learning rate
beta = 0.9 #momentum parameter
n_epoch = 10000 #number of iterations

### Création d'une instance du MLP à une couche cachée :

In [ ]:
model = MLP(H)

### Création d'une instance de la descente de gradient avec moment

In [ ]:
optimizer = GradientDescentWithMomentum(model, beta, lr)

### Nous pouvons lancer l'optimisation :

In [ ]:
for i in range(n_epoch):
    
    #Forward Pass
    X0,X1,X2,O = model.forward(X)
    
    #Compute Loss
    [l, dl_dO] = multinoulliCrossEntropyLoss(O, y)
    
    #Print Loss and Classif Accuracy
    pred = np.argmax(O, axis=1)
    acc = (np.argmax(O, axis=1) == y).astype('float').sum()/N
    print('Iter {} | Training Loss = {} | Training Accuracy = {}%'.format(i,l,acc*100))

    #Backward Pass (Compute Gradient)
    optimizer.zero_gradients()
    model.backward(dl_dO, O, X2, X1, X0)
    
    #Update Parameters
    optimizer.step()
    
    
    
    if(np.mod(i,10)==0):
        #Plot decision boundary
        ax.cla()
        for i in range(C):
            x_c = X[(y==i).ravel(),:]
            plt.plot(x_c[:,0],x_c[:,1],style_per_class[i])
        plot_contours(ax, model, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
        plt.pause(0.5)

# Travail à effectuer
* Après avoir implémenté la fonction `backward`, vous devriez constater que l'apprentissage sur l'exemple jouet "fonctionne", c'est-à-dire que le coût diminue progressivement vers zéro et le taux de bonne classification atteint rapidement 100%.
* Vous remarquerez que les paramètres du MLP sont initialisés aléatoirement selon une distribution uniforme centrée en zéro. Remplacer cette initialisation en initialisant tous les paramètres à zéro. Afficher les valeurs des activations (`X1`, `X2`, `O`) et des dérivées (notamment `dl_dW3` et `dl_dW2`). Que constatez-vous ? Pourquoi ?
* Restaurer l'initialisation aléatoire des paramètres.
* Modifier l'implémentation pour obtenir un MLP à 2 couches cachées (rajouter une transformation affine et une ReLU au MLP actuel).
* (Optionnel) Modifier l'implémentation pour la rendre générique de telle sorte que le nombre de couches cachées soit un hyper-paramètre réglable.

# PARTIE 2 : Reconnaissance de chiffres manuscrits

Maintenant que vous disposez d'une implémentation fonctionnelle de l'optimisation des paramètres d'un MLP sur un exemple "jouet", vous allez pouvoir l'adapter à un cas réel (de petite taille pour pouvoir effectuer des apprentissages en quelques dizaines de secondes sur CPU).  

La base de données étiquetées MNIST contient 70000 images de chiffres manuscrits. Chaque image est de taille 28x28, et est accompagnée d'une étiquette indiquant le chiffre qu'elle représente (0,1,2,..,9). Ainsi la base de données MNIST est très pratique pour résoudre un problème d'apprentissage supervisé consistant à entraîner un réseau de neurones à reconnaître des chiffres manuscrits.

### Mise en place de la base MNIST

+ Télécharger la base MNIST [ici](https://gbourmaud.github.io/files/intro_deep_learning/TP/TP_MLP/MNIST_bmp.zip)
+ Décompresser le ficher dans `/tmp` **(attention bien décompresser dans `/tmp` et pas dans `/net/.../tmp`)**
+ Les fichiers décompressés sont au format .bmp, ainsi vous pouvez les ouvrir pour regarder à quoi ressemble les données de cette base. **Un problème d'apprentissage supervisé repose sur une base de données étiquetées, ainsi il est très important de prendre le temps de visualiser les données et leurs étiquettes pour savoir ce qu'on s'apprête à traiter !**
+ Le code suivant permet de charger la base d'entraînement de MNIST (60000 images) ainsi que la base de test de MNIST (10000 images) (la partie `__main__` permet d'afficher quelques images et leurs étiquettes) :

In [ ]:
import PIL.Image as Image
from os import path
from os import listdir
import numpy as np
import math
import matplotlib.pyplot as plt

def load_MNIST(path_MNIST_bmp):
    
    filenames = listdir(path_MNIST_bmp)
    
    imgs_train = np.zeros((60000,28,28))
    labels_train = np.zeros(60000,dtype=np.uint8)
    n_train = 0
    
    imgs_test = np.zeros((10000,28,28))
    labels_test = np.zeros(10000,dtype=np.uint8)
    n_test = 0
    
    W=H=28
    for i, filename in enumerate(filenames):
        
        f_name, f_ext = path.splitext(filename)
        set_type, class_type ,num_im = f_name.split('_')
        
        num_im = int(num_im)
        img = np.array(Image.open(path.join(path_MNIST_bmp,filename)))/255.
        
        N_W = math.ceil(math.sqrt(num_im))
        N_H = math.ceil(num_im/N_W)
        
        im_array_ext = img.reshape(N_H,H,W*N_W).transpose((1,0,2))
        im_array_ext = im_array_ext.reshape(H,N_W*N_H,W).transpose((1,0,2))

        im_array = im_array_ext[:num_im,:,:]        
        
        if(set_type=='train'):
            imgs_train[n_train:n_train+num_im,:,:] = im_array
            labels_train[n_train:n_train+num_im] = int(class_type)
            n_train += num_im
        elif(set_type=='test'):
            imgs_test[n_test:n_test+num_im,:,:] = im_array
            labels_test[n_test:n_test+num_im] = int(class_type)
            n_test += num_im
                
    assert n_test == 10000
    assert n_train == 60000
        
        
    
    return imgs_train, labels_train, imgs_test, labels_test
    
if(__name__ == "__main__"):
    path_MNIST_bmp = './MNIST_bmp'

    imgs_train, labels_train, imgs_test, labels_test = load_MNIST(path_MNIST_bmp)
    n_train = imgs_train.shape[0]
    n_test = imgs_test.shape[0]
    
    ids = np.random.permutation(n_train)
    
    plt.figure()
    for i in range(8):
        for j in range(4):
            plt.subplot(4,8,i+1 + j*8)
            plt.imshow(imgs_train[ids[i+j*8],:,:])
            plt.title(labels_train[ids[i+j*8]])
            plt.axis('off')

    vec_train = imgs_train.reshape((n_train,-1))

# Travail à effectuer

+ Adapter le code de la Partie 1 pour entraîner un MLP à reconnaître des chiffres manuscrits avec une descente de gradient **stochastique** avec **arrêt prématuré** (la fonction `np.random.permutation` permet notamment de mélanger des indices, ce qui est pratique pour mélanger les données avant chaque epoch). **Remarque : Tout ce qui concerne la génération des données de l'exemple jouet de la Partie 1, ainsi que les fonctions d'affichage des frontières de décisions ne doivent pas être conservées.**
+ Si votre code fonctionne correctement, en quelques epochs (par exemple une dizaine d'epochs avec une taille de minibatch de 256), vous devriez obtenir un taux de bonne classification supérieur à 90% sur la base de test.
+ Modifier les hyperparamètres (pas d'apprentissage, taille du minibatch, nombre de couches du MLP, taille de chaque couche,...) pour essayer d'améliorer le taux de bonne classification.